In [3]:
import numpy as np
import pandas as pd
import keras
import tensorflow

import cv2
from keras.preprocessing.image import ImageDataGenerator 
from keras.models import Model
from keras.layers import Input,Dense,Conv2D,AveragePooling2D,Flatten,MaxPooling2D,BatchNormalization,Dropout

In [4]:
train_gen = ImageDataGenerator(featurewise_center = True,
                                  featurewise_std_normalization = True,
                                  samplewise_center = False,
                                  rotation_range = 45,width_shift_range=0.4 ,
                                  height_shift_range = 0.4,brightness_range=[-0.2,1.2],shear_range=4,rescale=1./255
                                  )
test_gen = ImageDataGenerator(rescale=1./255)

In [5]:
final_train_gen = train_gen.flow_from_directory(r"C:\Users\LAKSHMI NARASIMHARAO\innomatics\Deep Learning\Projects\kaggle (mask detection)\Train",
                              target_size=(227,227),class_mode='sparse',batch_size=90)

final_test_gen = test_gen.flow_from_directory(r"C:\Users\LAKSHMI NARASIMHARAO\innomatics\Deep Learning\Projects\kaggle (mask detection)\Validation",
                              target_size=(227,227),class_mode='sparse',batch_size=90)

Found 10000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.


In [15]:
input_layer = Input(shape=(227, 227, 3))

conv1 = Conv2D(96, (11,11), strides=(4, 4), padding='valid', activation='relu')(input_layer)
max_pool1 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='valid')(conv1)
bn1 = BatchNormalization()(max_pool1)

conv2 = Conv2D(256, (5, 5), strides=(1, 1), padding='same', activation='relu')(bn1)
max_pool2 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='valid')(conv2)
bn2 = BatchNormalization()(max_pool2)


conv3 = Conv2D(384, (3, 3), strides=(1, 1), padding='same', activation='relu')(bn2)
bn3 = BatchNormalization()(conv3)

conv4 = Conv2D(384, (3, 3), strides=(1, 1), padding='same', activation='relu')(bn3)
bn4 = BatchNormalization()(conv4)

conv5 = Conv2D(256, (3, 3), strides=(1, 1), padding='same', activation='relu')(bn4)
max_pool3 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='valid')(conv5)
bn5 = BatchNormalization()(max_pool3)


flatten = Flatten()(bn5)

hidd1 = Dense(9216, activation='relu')(flatten)
hidd2 = Dense(4096, activation='relu')(hidd1)
d1 = Dropout(rate=0.5)(hidd2)
hidd3 = Dense(4096, activation='relu')(d1)
d2 = Dropout(rate=0.5)(hidd3)

output_layer = Dense(2, activation='softmax')(d2)

In [16]:
atex_model = Model(inputs = input_layer,outputs = output_layer)

In [17]:
atex_model.summary()

Model: "functional_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 227, 227, 3)]     0         
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 55, 55, 96)        34944     
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 27, 27, 96)        0         
_________________________________________________________________
batch_normalization_30 (Batc (None, 27, 27, 96)        384       
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 27, 27, 256)       614656    
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 13, 13, 256)       0         
_________________________________________________________________
batch_normalization_31 (Batc (None, 13, 13, 256)     

In [18]:
atex_model.compile(optimizer='adam',loss = 'sparse_categorical_crossentropy',metrics=['accuracy'])

atex_model.fit_generator(final_train_gen,steps_per_epoch=10000//90,epochs=10,
                         validation_data=final_test_gen,validation_steps=800//90)

Epoch 1/10
111/111 [==============================] - 985s 9s/step - loss: 3.3288 - accuracy: 0.6911 - val_loss: 0.4907 - val_accuracy: 0.7819
Epoch 2/10
111/111 [==============================] - 883s 8s/step - loss: 0.4934 - accuracy: 0.7663 - val_loss: 0.5715 - val_accuracy: 0.6889
Epoch 3/10
111/111 [==============================] - 1067s 10s/step - loss: 0.4744 - accuracy: 0.7775 - val_loss: 0.4605 - val_accuracy: 0.7708
Epoch 4/10
111/111 [==============================] - 1185s 11s/step - loss: 0.4611 - accuracy: 0.7907 - val_loss: 0.2428 - val_accuracy: 0.9333
Epoch 5/10
111/111 [==============================] - 1180s 11s/step - loss: 0.4523 - accuracy: 0.7911 - val_loss: 0.4068 - val_accuracy: 0.8861
Epoch 6/10
111/111 [==============================] - 1000s 9s/step - loss: 0.4223 - accuracy: 0.8000 - val_loss: 0.2849 - val_accuracy: 0.8917
Epoch 7/10
111/111 [==============================] - 883s 8s/step - loss: 0.4149 - accuracy: 0.8040 - val_loss: 0.6772 - val_accuracy:

In [24]:
test_loss, test_accuracy = atex_model.evaluate(final_test_gen)

print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)


9/9 [==============================] - 11s 1s/step - loss: 0.2244 - accuracy: 0.9175
Test Loss: 0.22442694008350372
Test Accuracy: 0.9175000190734863


In [25]:
atex_model.save('final_model.h5')

In [4]:
my_model = tensorflow.keras.models.load_model("final_model.h5")

In [5]:
final_train_gen.class_indices

{'WithMask': 0, 'WithoutMask': 1}

In [43]:
import os
for file_name in os.listdir(r'C:\Users\LAKSHMI NARASIMHARAO\innomatics\Deep Learning\Projects\kaggle (mask detection)\Test\WithMask'):
    print(file_name)
    break

1163.png


In [6]:
img = cv2.imread(r"C:\Users\LAKSHMI NARASIMHARAO\innomatics\Deep Learning\Projects\kaggle (mask detection)\Test\WithMask\152.png")

np.argmax(my_model.predict(cv2.resize(img,(227,227))[np.newaxis]))

0

In [7]:
face_model = cv2.CascadeClassifier(r"C:\Users\LAKSHMI NARASIMHARAO\innomatics\Deep Learning\opencv\haarcascade_frontalface_alt.xml")
for x,y,w,h in face_model.detectMultiScale(img):
    if np.argmax(my_model.predict(cv2.resize(img,(227,227))[np.newaxis])) == 0:
        cv2.putText(img,'Mask',(x+10,y+10),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0))
    else:
        cv2.putText(img,' No Mask',(x+10,y+10),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0))


In [8]:
cv2.imshow('img',img)
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
import os
count=0
for file_name in os.listdir(r'C:\Users\LAKSHMI NARASIMHARAO\innomatics\Deep Learning\Projects\kaggle (mask detection)\Test\WithMask'):
    img = cv2.imread(r"C:\Users\LAKSHMI NARASIMHARAO\innomatics\Deep Learning\Projects\kaggle (mask detection)\Test\WithMask\{}".format(file_name))
    face_model = cv2.CascadeClassifier(r"C:\Users\LAKSHMI NARASIMHARAO\innomatics\Deep Learning\opencv\haarcascade_frontalface_alt.xml")
    for x,y,w,h in face_model.detectMultiScale(img):
        if np.argmax(my_model.predict(cv2.resize(img,(227,227))[np.newaxis])) == 0:
            cv2.putText(img,'Mask',(x+10,y+10),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0))
        else:
            cv2.putText(img,' No Mask',(x+10,y+10),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0))
    if count<10:
        cv2.imshow('img',img)
        cv2.waitKey()
        cv2.destroyAllWindows()
        count+=1


#### VGG19 and VGG16

In [4]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import VGG16, VGG19
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Constants
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
NUM_CLASSES = 2

# Load pre-trained models
base_model_vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model_vgg19 = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers
for layer in base_model_vgg16.layers:
    layer.trainable = False

for layer in base_model_vgg19.layers:
    layer.trainable = False

# Create model architecture
model_vgg16 = Sequential([
    base_model_vgg16,
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(NUM_CLASSES, activation='softmax')
])

model_vgg19 = Sequential([
    base_model_vgg19,
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(NUM_CLASSES, activation='softmax')
])

# Compile the models
model_vgg16.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_vgg19.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Data preprocessing
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    r"C:\Users\LAKSHMI NARASIMHARAO\innomatics\Deep Learning\Projects\kaggle (mask detection)\Train",
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        r"C:\Users\LAKSHMI NARASIMHARAO\innomatics\Deep Learning\Projects\kaggle (mask detection)\Validation",
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical')

# Train the models
history_vgg16 = model_vgg16.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE
)

# Evaluate the models
test_generator = test_datagen.flow_from_directory(
    r"C:\Users\LAKSHMI NARASIMHARAO\innomatics\Deep Learning\Projects\kaggle (mask detection)\Test",
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False)

Found 10000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.
Epoch 1/10
312/312 [==============================] - 2237s 7s/step - loss: 0.2436 - accuracy: 0.9614 - val_loss: 0.0182 - val_accuracy: 0.9912
Epoch 2/10
312/312 [==============================] - 3324s 11s/step - loss: 0.0343 - accuracy: 0.9884 - val_loss: 0.1142 - val_accuracy: 0.9500
Epoch 3/10
312/312 [==============================] - 3118s 10s/step - loss: 0.0327 - accuracy: 0.9888 - val_loss: 0.0269 - val_accuracy: 0.9925
Epoch 4/10
312/312 [==============================] - 2350s 8s/step - loss: 0.0265 - accuracy: 0.9899 - val_loss: 0.0188 - val_accuracy: 0.9962
Epoch 5/10
312/312 [==============================] - 2237s 7s/step - loss: 0.0277 - accuracy: 0.9896 - val_loss: 0.0079 - val_accuracy: 0.9975
Epoch 6/10
312/312 [==============================] - 3428s 11s/step - loss: 0.0238 - accuracy: 0.9922 - val_loss: 0.0070 - val_accuracy: 0.9987
Epoch 7/10
312/312 [=============================

KeyboardInterrupt: 

history_vgg19 = model_vgg19.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE
)

In [11]:
model_vgg16.save('mask_detection_vgg16.h5')

### Model Evaluation and Testing

In [10]:
train_generator.class_indices

{'WithMask': 0, 'WithoutMask': 1}

In [6]:
# Evaluate the models
test_generator = test_datagen.flow_from_directory(
    r"C:\Users\LAKSHMI NARASIMHARAO\innomatics\Deep Learning\Projects\kaggle (mask detection)\Test",
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False)

Found 992 images belonging to 2 classes.


In [7]:
vgg16_scores = model_vgg16.evaluate(test_generator)
print("VGG16 Test Loss:", vgg16_scores[0])
print("VGG16 Test Accuracy:", vgg16_scores[1])

31/31 [==============================] - 192s 6s/step - loss: 0.0086 - accuracy: 0.9980
VGG16 Test Loss: 0.008588694036006927
VGG16 Test Accuracy: 0.9979838728904724


In [8]:
vgg16_scores = model_vgg16.evaluate(validation_generator)

25/25 [==============================] - 152s 6s/step - loss: 0.0050 - accuracy: 0.9975


NameError: name 'vgg19_scores' is not defined

In [9]:
print("VGG16 validation Loss:", vgg16_scores[0])
print("VGG16 validation Accuracy:", vgg16_scores[1])

VGG16 validation Loss: 0.0050130062736570835
VGG16 validation Accuracy: 0.9975000023841858


In [6]:
final_model = tensorflow.keras.models.load_model("mask_detection_vgg16.h5")

In [12]:
img = cv2.imread(r"C:\Users\LAKSHMI NARASIMHARAO\innomatics\Deep Learning\Projects\kaggle (mask detection)\Test\WithMask\3.png")

np.argmax(final_model.predict(cv2.resize(img,(224,224),interpolation=cv2.INTER_AREA)[np.newaxis]))

0

In [17]:
face_model = cv2.CascadeClassifier(r"C:\Users\LAKSHMI NARASIMHARAO\innomatics\Deep Learning\opencv\haarcascade_frontalface_alt.xml")
for x,y,w,h in face_model.detectMultiScale(img):
    if np.argmax(final_model.predict(cv2.resize(img,(224,224))[np.newaxis])) == 0:
        cv2.putText(img,'Mask',(x+10,y+10),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0))
    else:
        cv2.putText(img,' No Mask',(x+10,y+10),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0))


In [20]:
cv2.imshow('img',cv2.resize(img,(224,224),interpolation=cv2.INTER_LINEAR))
cv2.waitKey()
cv2.destroyAllWindows()